In [1]:
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import mysql.connector

db = mysql.connector.connect(host = "localhost",
                             username = "root",
                             password = "M@ngo112",
                             database = "ECommerce")

cur = db.cursor()

ModuleNotFoundError: No module named 'mysql'

# List all the unique cities where customers are located.

In [ ]:
query = """ SELECT DISTINCT customer_city from customers """

cur.execute(query)

data = cur.fetchall()

data

# Count the numbers of orders placed in 2017.

In [ ]:
query = """ SELECT COUNT(order_id) FROM orders WHERE year(order_purchase_timestamp) = 2017  """

cur.execute(query)

data = cur.fetchall()

"Total orders placed in 2017 are", data[0][0]

('Total orders placed in 2017 are', 45101)